# Impact of the borrower characteristics on their loan interest rates
## by Jose Sotelo

## Investigation Overview

In this investigation, I wanted to look at the characteristics of borrowers that could affect their interest rate. The main focus was on the Credit Grade, income range, if the borrower is a home owner or not and the borrower employment status.


## Dataset Overview

The data consisted of borrower features of approximately 113,937 loans. The attributes included employment data like the current status and for how long the borrowe have been employed, as well as additional data such as credit grade, income range, etc. Sixty-six data points were removed from the analysis because they weren't going to be used in this analysis.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [ ]:
# load in the dataset into a pandas dataframe
df = pd.read_csv('../input/prosper-loan/prosperLoanData.csv')

# create a copy so we can made some modifications
df_main = df.copy()

In [ ]:
# Only leave the columns that are going to be explored
df_main = df_main[['ListingKey', 
               'CreditGrade', 
               'Term', 
               'LoanStatus', 
               'BorrowerAPR', 
               'BorrowerRate', 
               'ProsperScore', 
               'ListingCategory (numeric)', 
               'BorrowerState', 
               'Occupation', 
               'EmploymentStatus', 
               'EmploymentStatusDuration', 
               'IsBorrowerHomeowner', 
               'IncomeRange',
               'TotalProsperLoans'
              ]]

In [ ]:
# convert credit grades and income range into ordered categorical types
# Also, created a variable with the base color (blue) for easy access
base_color = sb.color_palette()[0]

ordinal_var_dict = {'CreditGrade': ['HR', 'E', 'D', 'C', 'B', 'A', 'AA'],
                    'IncomeRange': ['$100,000+', '$75,000-99,999', '$50,000-74,999', '$25,000-49,999', '$1-24,999'],
                    'EmploymentStatus': ['Full-time', 'Part-time', 'Self-employed', 'Retired', 'Not Employed']
                   }

for var in ordinal_var_dict:
    ordered_var = pd.api.types.CategoricalDtype(ordered = True,
                                                categories = ordinal_var_dict[var])
    df_main[var] = df_main[var].astype(ordered_var)

In [ ]:
# Dropping duplicated Listing Keys
df_main.drop_duplicates(subset=['ListingKey'], inplace=True)

## Distribution of the Borrower Rate

Most of the borrowers interest rates tend to be between 0.1 and 0.2, but when we reduce the bin size to 0.01 we can notice that at 0.31 there is a big spike.

In [ ]:
binsize = 0.01
bins = np.arange(0, df_main['BorrowerRate'].max()+binsize, binsize)

plt.figure(figsize=[8, 5])
plt.hist(data = df_main, x = 'BorrowerRate', bins = bins)
plt.xlabel('Borrower Interest Rate')
plt.show()

## Impact of the Credit Grade on the loan interest rate

As expected, the higher the Credit Grade the lower the Borrower interest rate tends to be.

In [ ]:
plt.figure(figsize = [16, 5]);
plt.title('Relationship between the Credit Grade and the Borrower Interest Rate');
sb.violinplot(data=df_main, x='CreditGrade', y='BorrowerRate', color=base_color);
plt.xlabel('Credit Grade');
plt.ylabel('Interest Rate');

## Impact of the Employment Status on the loan interest rate

Borrowers that are employed full-time or part-time can have a lower interest rate than a self-employed, which means that working for a company is more beneficial (when asking for a loan at least) than being an entrepeneur or a freelancer.

In [ ]:
plt.figure(figsize = [10, 5])

plt.title('Relationship between the Employment Status and the Borrower Rate')
plt.xticks(rotation=90)
sb.boxplot(data=df_main, x='EmploymentStatus', y='BorrowerRate', color=base_color);

## Impact of the Income Range on the loan interest rate

As expected, the higher the income the lower the Borrower interest rate tend to be.

In [ ]:
boxen = sb.catplot(data=df_main, x='IncomeRange', y='BorrowerRate', kind='boxen', height=5, aspect=16/5, color=base_color);
boxen.fig.subplots_adjust(top=0.9);
boxen.fig.suptitle('Income Range vs Borrower Interest Rate');
boxen.set_xlabels('Income Range');
boxen.set_ylabels('Borrower Interest Rate');

## Impact of owning a home on the loan interest rate

Home owners tend to have a lower Borrower interest rate, also, the maximum interest rate is lower than the maximum of borrowers that do not own a home.

In [ ]:
plt.title('Relationship between being a home owner and the Borrower Interest Rate')
sb.boxplot(data=df_main, x='IsBorrowerHomeowner', y='BorrowerRate');
plt.xlabel('Borrower is a Home Owner');
plt.ylabel('Borrower Interest Rate');

# The Credit Grade is more important than the income

It's clear that the income rate isn't as powerful as the Credit Grade, as even when having the highest income ($100,000+), the borrower will have a high interest rate if the Credit Grade is HR.

In [ ]:
g = sb.FacetGrid(df_main, col="IncomeRange",  row="CreditGrade", margin_titles=True);
g.map(sb.histplot, "BorrowerRate");
g.fig.subplots_adjust(top=0.9);
g.fig.suptitle('Relationship between the Credit Grade, the Income Range and the Borrower Interest Rate');

# The Credit Grade is more important than owning a house

Similar to the income rate, being a home owner isn't as powerful as the Credit Grade, as even when the borrower is a home owner, the borrower will have a high interest rate if the Credit Grade is HR.

That being said, most of borrowers with credit grade HR do not own houses, and we see the same the other way around, most of the borrowers with credit grade of AA are home owners.

In [ ]:
h = sb.FacetGrid(df_main, col="CreditGrade",  row="IsBorrowerHomeowner", margin_titles=True);
h.map(sb.histplot, "BorrowerRate");
h.fig.subplots_adjust(top=0.9);
h.fig.suptitle('Relationship between the Credit Grade, the Borrower Interest Rate and being a home owner');